## Different Classification with Python

### *By Rohit Kumar Singh and Bavalpreet Singh*

<a href="https://www.bigdatauniversity.com"><img src="https://ibm.box.com/shared/static/cw2c7r3o20w9zn8gkecaeyjhgw3xdgbj.png" width="400" align="center"></a>

<h1 align="center"><font size="5">Classification with Python</font></h1>

In this notebook we try to practice all the classification algorithms that we learned in this course.

We load a dataset using Pandas library, and apply the following algorithms, and find the best one for this specific dataset by accuracy evaluation methods.

Lets first load required libraries:

In [1]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

<img src="https://miro.medium.com/fit/c/1730/520/1*glrB0KgjOcTiKUEx7T8tcA.png" width="600" align="center">

### About dataset

This dataset is about past loans. The __Loan_train.csv__ data set includes details of 346 customers whose loan are already paid off or defaulted. It includes following fields:

| Field          | Description                                                                           |
|----------------|---------------------------------------------------------------------------------------|
| Loan_status    | Whether a loan is paid off on in collection                                           |
| Principal      | Basic principal loan amount at the                                                    |
| Terms          | Origination terms which can be weekly (7 days), biweekly, and monthly payoff schedule |
| Effective_date | When the loan got originated and took effects                                         |
| Due_date       | Since it’s one-time payoff schedule, each loan has one single due date                |
| Age            | Age of applicant                                                                      |
| Education      | Education of applicant                                                                |
| Gender         | The gender of applicant                                                               |

Lets download the dataset

In [1]:
!wget -O loan_train.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_train.csv

### Load Data From CSV File  

In [1]:
df = pd.read_csv('loan_train.csv')
df.head().style.background_gradient(cmap='RdGy')

In [1]:
df.shape

### Convert to date time object 

In [1]:
df['due_date'] = pd.to_datetime(df['due_date'])
df['effective_date'] = pd.to_datetime(df['effective_date'])
df.head().style.background_gradient(cmap='RdGy')

# Data visualization and pre-processing



Let’s see how many of each class is in our data set 

In [1]:
df['loan_status'].value_counts()

260 people have paid off the loan on time while 86 have gone into collection 


Lets plot some columns to underestand data better:

In [1]:
import seaborn as sns

bins = np.linspace(df.Principal.min(), df.Principal.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'Principal', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

In [1]:
bins = np.linspace(df.age.min(), df.age.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'age', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

# Pre-processing:  Feature selection/extraction

### Lets look at the day of the week people get the loan 

In [1]:
df['dayofweek'] = df['effective_date'].dt.dayofweek
bins = np.linspace(df.dayofweek.min(), df.dayofweek.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'dayofweek', bins=bins, ec="k")
g.axes[-1].legend()
plt.show()


We see that people who get the loan at the end of the week dont pay it off, so lets use Feature binarization to set a threshold values less then day 4 

In [1]:
df['weekend'] = df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
df.head().style.background_gradient(cmap='RdGy')

## Convert Categorical features to numerical values

Lets look at gender:

In [1]:
df.groupby(['Gender'])['loan_status'].value_counts(normalize=True)

86 % of female pay there loans while only 73 % of males pay there loan


Lets convert male to 0 and female to 1:


In [1]:
df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
df.head().style.background_gradient(cmap='RdGy')

## One Hot Encoding  
#### How about education?

In [1]:
df.groupby(['education'])['loan_status'].value_counts(normalize=True)

#### Feature befor One Hot Encoding

In [1]:
df[['Principal','terms','age','Gender','education']].head().style.background_gradient(cmap='RdGy')

#### Use one hot encoding technique to conver categorical varables to binary variables and append them to the feature Data Frame 

In [1]:
Feature = df[['Principal','terms','age','Gender','weekend']]
Feature = pd.concat([Feature,pd.get_dummies(df['education'])], axis=1)
Feature.drop(['Master or Above'], axis = 1,inplace=True)
Feature.head().style.background_gradient(cmap='RdGy')


### Feature selection

Lets defind feature sets, X:

In [1]:
X = Feature
X1 = X
X[0:5]

In [1]:
print(X.columns)

What are our lables?

In [1]:
y = df['loan_status'].values
y[0:5]

## Normalize Data 

Data Standardization give data zero mean and unit variance (technically should be done after train test split )

In [1]:
X= preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

# Classification 

Now, it is your turn, use the training set to build an accurate model. Then use the test set to report the accuracy of the model
You should use the following algorithm:
- K Nearest Neighbor(KNN)
- Decision Tree
- Support Vector Machine
- Logistic Regression



__ Notice:__ 
- You can go above and change the pre-processing, feature selection, feature-extraction, and so on, to make a better model.
- You should use either scikit-learn, Scipy or Numpy libraries for developing the classification algorithms.
- You should include the code of the algorithm in the following cells.

In [1]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

# K Nearest Neighbor(KNN)
Notice: You should find the best k to build the model with the best accuracy.  
**warning:** You should not use the __loan_test.csv__ for finding the best k, however, you can split your train_loan.csv into train and test to find the best __k__.

In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
Ks = 10
mean_acc = np.zeros((Ks-1))
std_acc = np.zeros((Ks-1))
ConfustionMx = [];
for n in range(1,Ks):
    #Train Model and Predict  
    neigh = KNeighborsClassifier(n_neighbors = n).fit(X_train,y_train)
    yhat=neigh.predict(X_test)
    mean_acc[n-1] = metrics.accuracy_score(y_test, yhat)
    std_acc[n-1]=np.std(yhat==y_test)/np.sqrt(yhat.shape[0])
mean_acc
#Plot model accuracy for Different number of Neighbors
plt.plot(range(1,Ks),mean_acc,'g')
plt.fill_between(range(1,Ks),mean_acc - 1 * std_acc,mean_acc + 1 * std_acc, alpha=0.10)
plt.legend(('Accuracy ', '+/- 3xstd'))
plt.ylabel('Accuracy ')
plt.xlabel('Number of Neighbors (K)')
plt.tight_layout()
plt.show()
print( "The best accuracy was with", mean_acc.max(), "with k=", mean_acc.argmax()+1)


In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
k = 7 # For best K
#Train Model and Predict  
neigh = KNeighborsClassifier(n_neighbors = k).fit(X_train,y_train)
yhat = neigh.predict(X_test)
print("Train set Accuracy: ", metrics.accuracy_score(y_train, neigh.predict(X_train)))
print("Test set Accuracy: ", metrics.accuracy_score(y_test, yhat))

# Decision Tree

In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
drugTree = DecisionTreeClassifier(criterion="entropy", max_depth = 4)
drugTree # it shows the default parameters
drugTree.fit(X_train,y_train)
predTree = drugTree.predict(X_test)

from sklearn import metrics
print("DecisionTrees's Accuracy: ", metrics.accuracy_score(y_test, predTree))

In [1]:
#INSTALLATIONS TO VIEW THE DECISION TREE
!conda install -c conda-forge pydotplus -y

In [1]:
from sklearn.externals.six import StringIO
import pydotplus
import matplotlib.image as mpimg
from sklearn import tree

%matplotlib inline 
dot_data = StringIO()
filename = "drugtree.png"
featureNames = df.columns[3: 11]
targetNames = df["loan_status"].unique().tolist()
out=tree.export_graphviz(drugTree,feature_names=featureNames, out_file=dot_data, class_names= np.unique(y_train), filled=True,  special_characters=True,rotate=False)  
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png(filename)
img = mpimg.imread(filename)
plt.figure(figsize=(100, 200))
plt.imshow(img,interpolation='nearest')

# Support Vector Machine

### **For visualization purpose, using linear kernel for SVM.**

In [1]:
from sklearn import svm
from sklearn import metrics
clf2 = svm.SVC(kernel='linear')
clf2.fit(X_train, y_train) 
yhat2 = clf2.predict(X_test)
print("SVM ", metrics.accuracy_score(y_test, yhat2))

In [1]:
clf2.support_vectors_

In [1]:
print(X1.columns)

In [1]:
# Determining the most contributing features for SVM classifier

pd.Series(abs(clf2.coef_[0]), index=X1.columns).nlargest(10).plot(kind='barh',figsize=(8, 6))

In [1]:
clf2 = svm.SVC(kernel='rbf')
clf2.fit(X_train, y_train) 
yhat2 = clf2.predict(X_test)
print("SVM ", metrics.accuracy_score(y_test, yhat2))

# Logistic Regression

In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import log_loss
from sklearn.metrics import jaccard_similarity_score
LR = LogisticRegression(C=0.01, solver='liblinear').fit(X_train,y_train)
yhat = LR.predict(X_test)
yhat_prob = LR.predict_proba(X_test)
print("Logistic Regression's Log Loss Accuracy: ", log_loss(y_test, yhat_prob))
print("Logistic Regression's Jaccard Similarity Accuracy: ", jaccard_similarity_score(y_test, yhat))


# Model Evaluation using Test set

In [1]:
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

First, download and load the test set:

In [1]:
!wget -O loan_test.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_test.csv

### Load Test set for evaluation 

In [1]:
test_df = pd.read_csv('loan_test.csv')
test_df.head().style.background_gradient(cmap='RdGy')

In [1]:
# Preprocessing the test data set
test_df['due_date'] = pd.to_datetime(test_df['due_date'])
test_df['effective_date'] = pd.to_datetime(test_df['effective_date'])
test_df['loan_status'].value_counts()
test_df['dayofweek'] = test_df['effective_date'].dt.dayofweek
test_df['weekend'] = test_df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
test_df.groupby(['Gender'])['loan_status'].value_counts(normalize=True)
test_df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
test_df.groupby(['education'])['loan_status'].value_counts(normalize=True)
test_df[['Principal','terms','age','Gender','education']].head()
test_df[['Principal','terms','age','Gender','education']].head()
Feature = test_df[['Principal','terms','age','Gender','weekend']]
Feature = pd.concat([Feature,pd.get_dummies(test_df['education'])], axis=1)
Feature.drop(['Master or Above'], axis = 1,inplace=True)
X = Feature
y_test = test_df['loan_status'].values
X_test= preprocessing.StandardScaler().fit(X).transform(X)

In [1]:
# K Nearest Neighbor(KNN) Prediction
yhat = neigh.predict(X_test)
print("KNN's Jaccard Similarity Accuracy: %.2f" % jaccard_similarity_score(y_test, yhat))
print("KNN Avg F1-score: %.2f" % f1_score(y_test, yhat, average='weighted'))

In [1]:
# Support Vector Machine Prediction
yhat2 = clf2.predict(X_test)
print("SVM's Jaccard Similarity Accuracy: %.2f" % jaccard_similarity_score(y_test, yhat2))
print("SVM Avg F1-score: %.2f" % f1_score(y_test, yhat2, average='weighted'))


In [1]:
# Logistic Regression Prediction
yhat3 = LR.predict(X_test)
yhat_prob3 = LR.predict_proba(X_test)
print("Logistic Regression's Jaccard Similarity Accuracy:  %.2f" % jaccard_similarity_score(y_test, yhat3))
print("LRP Avg F1-score: %.2f" % f1_score(y_test, yhat3, average='weighted'))
print("Logistic Regression's Log Loss Accuracy: %.2f" % log_loss(y_test, yhat_prob3))

In [1]:
# Decision Tree Prediction
predTree = drugTree.predict(X_test)
print("DecisionTrees's Jaccard Similarity Accuracy: %.2f" % jaccard_similarity_score(y_test, predTree))
print("DecisionTrees Avg F1-score: %.2f" % f1_score(y_test, predTree, average='weighted'))

# Report
You should be able to report the accuracy of the built model using different evaluation metrics:

| Algorithm          | Jaccard | F1-score | LogLoss |
|--------------------|---------|----------|---------|
| KNN                | 0.67    |  0.63    | NA      |
| Decision Tree      | 0.72    | 0.74     | NA      |
| SVM                | 0.80    | 0.76     | NA      |
| LogisticRegression | 0.74    | 0.66     | 0.57    |

After complete evaluation of the models using the 4 above said algorithms, the SVM - Support Vector Machine holds the highest accuracy rate of 80% and 76% in both the evaluation models say Jaccard Index and F1-Score.

<img src="https://d31bgfoj87qaaj.cloudfront.net/blog/wp-content/uploads/2019/04/Blog-Loan-Approved-Feature-min.jpg" width="500" align="center">

<h3>Thanks for completing this lesson!</h3>

Credit :-
<h4>Author of Solution and Visualization:  <a href="https://www.linkedin.com/in/rohit-kumar-singh1996/"> Rohit Kumar Singh </a> and  <a href="https://www.linkedin.com/in/bavalpreet-singh-a7b627172/"> Bavalpreet Singh </a></h4>
<h4>Author of Format:  <a href="https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a></h4>
<p>Copyright &copy; 2018 <a href="https://cocl.us/DX0108EN_CC">Cognitive Class</a>. This notebook and its source code are released under the terms of the <a href="https://bigdatauniversity.com/mit-license/">MIT License</a>.</p>